# 1. load/reload all dependency

In [4]:
import numpy as np
import pandas as pd
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

import sys
sys.path.append("../")
import pyhealth.datasets.datasets as datasets
import pyhealth.models.models as models
from importlib import reload
reload(datasets)
reload(models)
import torch
print(torch.cuda.is_available())


True


# 2. load dataset and initialize model

In [5]:
# load MIMIC-III
dataset = datasets.MIMIC_III()

# initialize the model and build the dataloaders
dataset.get_dataloader("RETAIN")
model = models.RETAIN(
    dataset=dataset,
    emb_dim=64,
)

loaded the med table!
loaded the diag table!
loaded the prod table!
generated .pat_to_visit!
generated .visit_dict!
----- preparing code mappings -----
source loaded from ./data cache
source loaded from ./data cache
source loaded from ./data cache
source loaded from ./data cache
mapping finished: RxNorm -> ATC4
mapping finished: ATC4 -> RxNorm
load time: 2.4476158618927s
-----------------------------------------
generated .encoded_visit_dict!
generated .maps (for code to index mappings)!
load severe ddi pairs from https://drive.google.com/uc?id=1R88OIhn-DbOYmtmVYICmjBSOIsEljJMh!
ddi info is from https://drive.google.com/file/d/1mnPc0O0ztz0fkv3HF-dpmBb8PLWsEoDz/view?usp=sharing!
generated train/val/test dataloaders for RETAIN model!


# 3. train and test

In [9]:
# set trainer with checkpoint
# checkpoint_callback = ModelCheckpoint(dirpath='./model_cpt')
# trainer = Trainer(
#     gpus=1,
#     max_epochs=1,
#     progress_bar_refresh_rate=5,
#     callbacks=[checkpoint_callback],
# )

# # train model
# trainer.fit(
#     model=model,
#     train_dataloaders=dataset.train_loader,
#     val_dataloaders=dataset.val_loader,
# )

# test the best model
model.summary(
    output_path='./',
    test_dataloaders=dataset.test_loader,
    ckpt_path=checkpoint_callback.best_model_path,
)

--- Test Summary ---
DDI rate: 0.07337
Jaccard: 0.4229
PRAUC: 0.7233
AVG_PRC: 0.6329
AVG_RECALL: 0.5985
AVG_F1: 0.5885
AVG_MED: 33.07



In [11]:
model.pat_info_test

[[[2215,
   451,
   460,
   191,
   382,
   466,
   7,
   192,
   2119,
   145,
   4018,
   201,
   8,
   180,
   465,
   50,
   2525,
   3204,
   19,
   2485,
   1874,
   2467,
   2786,
   578,
   385,
   1218,
   2380,
   391,
   413,
   409,
   407,
   337,
   136,
   6766,
   6659,
   6673,
   8614,
   8614,
   8614,
   8614,
   8614,
   8614,
   8614,
   8614,
   8614,
   8614,
   8614,
   8614],
  [108, 1, 15],
  [49,
   74,
   2,
   55,
   3,
   4,
   5,
   8,
   9,
   10,
   89,
   11,
   12,
   94,
   50,
   34,
   103,
   57,
   13,
   78,
   14,
   37,
   45,
   58,
   16,
   38,
   115,
   18,
   105,
   0,
   96,
   52,
   21,
   22,
   98,
   41,
   24,
   61,
   71,
   85,
   42,
   43,
   44,
   1,
   99,
   29,
   47,
   100,
   48,
   30,
   31],
  40000,
  129338,
  array([3.46774727e-01, 4.98642743e-01, 7.69051433e-01, 7.63909578e-01,
         5.81443131e-01, 1.42426923e-01, 3.11810583e-01, 4.15784210e-01,
         6.30244374e-01, 8.85129333e-01, 8.01250398e-01, 9.3

In [33]:
nested_dict = {}
for cur_pat in model.pat_info_test:
    for cur_visit in cur_pat:
        pat_id = cur_visit[3]
        visit_id = cur_visit[4]
        diag = model.maps['diag'].decodes(cur_visit[0])
        if -1 in diag: diag.remove(-1)
        prod = model.maps['prod'].decodes(cur_visit[1])
        if -1 in prod: prod.remove(-1)
        gt_med = model.maps['med'].decodes(cur_visit[2])
        if -1 in gt_med: gt_med.remove(-1)
        pre_logits = cur_visit[5]
        pre_med = np.where(pre_logits >= 0.5)[0]
        if pat_id not in nested_dict:
            nested_dict[pat_id] = {}
        nested_dict[pat_id][visit_id] = {
            "diagnoses": diag,
            "procedures": prod,
            "real_prescription": gt_med,
            "predicted_prescription": model.maps['med'].decodes(pre_med),
            "prediction_logits": {
                atc3: str(np.round(logit, 4)) for atc3, logit in zip(model.maps['med'].code_to_idx.keys(), pre_logits)
            }
        }

In [31]:
len(model.maps['diag'].code_to_idx)

6658